In [4]:
import pandas as pd
import numpy as 

In [3]:
csv_pfizer = 'source_data/COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Pfizer.csv'
pfizer_df = pd.read_csv(csv_pfizer)

NameError: name 'pd' is not defined